<a href="https://colab.research.google.com/github/Manvi1718/Emotion_Analysis/blob/main/Emotion_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["KAGGLE_USERNAME"] = "annmittal"
os.environ["KAGGLE_KEY"] ="b36a820c483a21bd0633e9b284bbc42c"

!kaggle datasets download -d shawon10/ckplus

 83% 3.00M/3.63M [00:00<00:00, 5.04MB/s]
100% 3.63M/3.63M [00:00<00:00, 5.04MB/s]


In [2]:
!unzip ckplus.zip

Archive:  ckplus.zip
  inflating: CK+48/anger/S010_004_00000017.png  
  inflating: CK+48/anger/S010_004_00000018.png  
  inflating: CK+48/anger/S010_004_00000019.png  
  inflating: CK+48/anger/S011_004_00000019.png  
  inflating: CK+48/anger/S011_004_00000020.png  
  inflating: CK+48/anger/S011_004_00000021.png  
  inflating: CK+48/anger/S014_003_00000028.png  
  inflating: CK+48/anger/S014_003_00000029.png  
  inflating: CK+48/anger/S014_003_00000030.png  
  inflating: CK+48/anger/S022_005_00000030.png  
  inflating: CK+48/anger/S022_005_00000031.png  
  inflating: CK+48/anger/S022_005_00000032.png  
  inflating: CK+48/anger/S026_003_00000013.png  
  inflating: CK+48/anger/S026_003_00000014.png  
  inflating: CK+48/anger/S026_003_00000015.png  
  inflating: CK+48/anger/S028_001_00000022.png  
  inflating: CK+48/anger/S028_001_00000023.png  
  inflating: CK+48/anger/S028_001_00000024.png  
  inflating: CK+48/anger/S029_001_00000017.png  
  inflating: CK+48/anger/S029_001_00000018.png  

In [3]:
import torch
from torch import nn
import torchvision
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import albumentations
from albumentations import pytorch as AT

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from PIL import Image

In [5]:
use_cuda= torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [6]:
device

device(type='cuda')

In [7]:
!pip install split-folders

In [8]:
import splitfolders

In [9]:
#Splitting dataset into train ,test and validation

original="/content/drive/MyDrive/pytorch/ck+/CK+48"
splitted="/content/drive/MyDrive/pytorch/ck+/CK+48 splitted"

splitfolders.ratio(original,output=splitted,seed=1337,ratio=(.75,.25),group_prefix=None)

Copying files: 981 files [07:53,  2.07 files/s]


In [35]:
#Converting image dataset in folder to readable csv file

import os
import csv
from PIL import Image
import pandas as pd  # Import the pandas library for creating DataFrames

#Train folder to csv
image_folder = "/content/drive/MyDrive/pytorch/ck+/CK+48 splitted/train_d"
image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg") or f.endswith(".png")]

data = []
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    emotion_label = image_file.split("_")[0]

    image = Image.open(image_path)
    image = image.convert("L")
    pixel_values = list(image.getdata())

    data.append((emotion_label, pixel_values))
train_df = pd.DataFrame(data, columns=["Emotion", "Pixels"])

csv_file = "train.csv"
train_df.to_csv(csv_file, index=False)

#Test folder to csv
image_folder = "/content/drive/MyDrive/pytorch/ck+/CK+48 splitted/test_d"
image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg") or f.endswith(".png")]

data = []
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    emotion_label = image_file.split("_")[0]

    image = Image.open(image_path)
    image = image.convert("L")
    pixel_values = list(image.getdata())

    data.append((emotion_label, pixel_values))
test_df = pd.DataFrame(data, columns=["Emotion", "Pixels"])

csv_file2 = "test.csv"
test_df.to_csv(csv_file2, index=False)


In [37]:
train_df = pd.read_csv("train.csv")

In [38]:
train_df.head(50)

,Emotion,Pixels
0,contempt,"[220, 219, 220, 219, 202, 153, 64, 49, 42, 38,..."
1,contempt,"[196, 142, 167, 157, 117, 98, 92, 104, 112, 12..."
2,contempt,"[196, 143, 168, 159, 119, 99, 92, 105, 111, 12..."
3,contempt,"[155, 91, 51, 51, 48, 46, 47, 48, 51, 59, 59, ..."
4,contempt,"[72, 73, 60, 50, 43, 46, 53, 61, 68, 79, 75, 6..."
5,contempt,"[121, 105, 88, 98, 108, 101, 117, 146, 162, 16..."
6,contempt,"[214, 214, 214, 214, 214, 221, 151, 51, 53, 61..."
7,contempt,"[69, 50, 53, 56, 64, 66, 68, 73, 85, 91, 93, 9..."
8,contempt,"[72, 74, 71, 44, 43, 38, 41, 45, 47, 47, 49, 6..."
9,contempt,"[164, 148, 142, 121, 119, 106, 86, 135, 139, 1..."


In [36]:
test_df = pd.read_csv("test.csv")
test_df.head(50)

,Emotion,Pixels
0,sadness,"[47, 27, 5, 1, 7, 10, 4, 1, 5, 7, 9, 11, 12, 1..."
1,sadness,"[38, 23, 21, 37, 43, 60, 50, 34, 29, 36, 51, 9..."
2,sadness,"[74, 41, 43, 39, 35, 31, 26, 22, 20, 17, 17, 1..."
3,sadness,"[27, 10, 7, 6, 8, 7, 14, 13, 18, 21, 22, 26, 2..."
4,sadness,"[199, 200, 199, 201, 179, 89, 62, 67, 63, 61, ..."
5,sadness,"[43, 19, 3, 2, 9, 9, 4, 3, 6, 10, 10, 12, 11, ..."
6,sadness,"[39, 31, 12, 0, 0, 0, 0, 1, 1, 0, 4, 33, 62, 9..."
7,sadness,"[21, 15, 0, 0, 0, 0, 0, 6, 24, 9, 60, 62, 46, ..."
8,sadness,"[155, 55, 12, 9, 6, 10, 18, 31, 46, 59, 71, 79..."
9,sadness,"[84, 84, 84, 81, 74, 66, 43, 35, 48, 69, 97, 1..."


In [39]:
class_names=["Angry", "Contempt", "Disgust", "Fear", "Happy", "Sadness", "Surprise"]

In [40]:
img=train_df["Pixels"][0]
img

'[220, 219, 220, 219, 202, 153, 64, 49, 42, 38, 36, 32, 37, 52, 74, 99, 120, 132, 142, 147, 147, 145, 149, 150, 154, 154, 151, 150, 143, 141, 139, 136, 131, 123, 104, 86, 69, 54, 42, 36, 35, 33, 33, 34, 37, 73, 159, 180, 219, 219, 217, 207, 172, 89, 44, 40, 37, 36, 35, 44, 64, 91, 115, 128, 142, 149, 151, 151, 152, 155, 155, 160, 165, 166, 164, 158, 145, 139, 143, 141, 140, 133, 126, 112, 93, 75, 57, 43, 36, 34, 33, 33, 37, 36, 92, 178, 219, 216, 215, 188, 124, 49, 40, 37, 34, 36, 47, 70, 97, 120, 129, 141, 151, 153, 156, 158, 160, 161, 164, 168, 175, 177, 174, 172, 156, 148, 149, 148, 143, 138, 133, 124, 114, 100, 82, 61, 45, 36, 34, 32, 33, 37, 42, 136, 217, 219, 204, 158, 70, 39, 36, 34, 37, 48, 69, 98, 119, 129, 135, 146, 155, 163, 164, 164, 166, 166, 169, 172, 175, 177, 177, 172, 166, 162, 159, 152, 144, 139, 135, 127, 121, 114, 102, 82, 60, 47, 37, 32, 32, 34, 38, 68, 218, 212, 169, 105, 37, 36, 35, 37, 49, 71, 92, 113, 125, 130, 139, 151, 159, 169, 170, 169, 169, 169, 171, 170, 

In [48]:
#Model using CNN Architechture

class CK48Model(nn.Module):
  #Blocks
  def __init__(self,input_shape,num_classes):                                ##input shape= 1 one color channel (grayscale for our image data)
    super().__init__()
    self.block_1=nn.Sequential(
        nn.Conv2d(in_channels = input_shape, out_channels = 32, kernel_size=(3,3), stride=1, padding=1, bias = False),
        nn.ReLU(),
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3,3), stride=1, padding=1, bias = False),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2), stride=2)
    )

    self.block_2=nn.Sequential(
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=(3,3), stride=1, padding=1, bias = False),
        nn.ReLU(),
        nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size=(3,3), stride=1, padding=1, bias = False),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2), stride=2)
    )

    self.block_3=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=256*9*9,out_features=num_classes,bias = False)
    )

  def forward(self,x):
    x=self.block_1(x)
    x=self.block_2(x)
    x=self.block_3(x)
    return x

In [49]:
#input shape= 1 ~ we have one color channel only
#num_classes= 7 ~ we have 7 catagories in our dataset(7 emotions)

## Model Building
model = CK48Model(input_shape=1, num_classes=7).to(device)
model

CK48Model(
  (block_1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_3): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=20736, out_features=7, bias=False)
  )
)